# Error Logging

Create ETL Log table

In [ ]:
-- ETL Batch table
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name= N'EtlBatch' AND XTYPE='U')
BEGIN
    CREATE TABLE [dbo].[EtlBatch] (
        [batchId] [uniqueidentifier] NOT NULL,
        [batchStart] [datetime] NOT NULL
    );
END

-- ETL Log table
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name= N'EtlLog' AND XTYPE='U')
BEGIN
    CREATE TABLE [dbo].[EtlLog] (
        [logId] [int] IDENTITY(1,1) NOT NULL,
        [batchId] [uniqueidentifier] NULL,
        [errorLevel] [int] NULL,
        [custom_message] [varchar](1000) NULL,
        [message] [varchar](max) NULL,
        [logCreated] [datetime] NULL
    );
END

-- ETL Log History table
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name= N'EtlLogHistory' AND XTYPE='U')
BEGIN
    CREATE TABLE [dbo].[EtlLogHistory] (
        [logId] [int] IDENTITY(1,1) NOT NULL,
        [batchId] [uniqueidentifier] NULL,
        [errorLevel] [int] NULL,
        [message] [varchar](max) NOT NULL,
        [logCreated] [datetime] NOT NULL,
        [logCompName] [varchar](50) NULL,
        [logUserName] [varchar](50) NULL
    );
END

Create stored procedure to run at beginning of ETL to manage logging

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[usp_ETL_Preprocess]
AS
BEGIN
	SET NOCOUNT ON;

	-- Create ETL Batch Id
	INSERT INTO [dbo].[EtlBatch] (batchId, batchStart) VALUES (NEWID(), GETDATE());

	-- Move logs to history
	INSERT INTO [dbo].[EtlLogHistory] (batchId, errorLevel, message, logCreated)
	SELECT batchId, errorLevel, message, logCreated
	FROM [dbo].[EtlLog]
	ORDER BY logId;

	-- Clear logs
	DELETE FROM [dbo].[EtlLog];

END


Create stored procedure to insert error logs into the ETL Log table

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[LogError]
	@hasBatchId bit = 0, @customMessage varchar(1000) = ''	
AS
BEGIN
	SET NOCOUNT ON;

	DECLARE @BATCH_ID uniqueidentifier;
	--DECLARE @hasBatchId bit = 0;

	IF @hasBatchId = 1
	BEGIN	
		SET @BATCH_ID = (SELECT TOP 1 batchId FROM [BCC_dwh].[dbo].[etlBatch] ORDER BY batchStart DESC);	
	END 

	INSERT INTO [BCC_dwh].[dbo].[EtlLog] (batchId, errorLevel, custom_message, message, logCreated)
	SELECT  @BATCH_ID AS [batchId']
		, ERROR_SEVERITY() AS [errorLevel]
		, @customMessage AS [custom_message]
		, 'ErrorNumber - ' + COALESCE(CAST(ERROR_NUMBER() AS varchar(200)), ' ')
			+ ' ; ErrorState - ' +  COALESCE(CAST(ERROR_STATE() AS varchar(200)), ' ')
			+ ' ; ErrorProcedure - ' +  COALESCE(CAST(ERROR_PROCEDURE() AS varchar(200)), ' ')
			+ ' ; ErrorLine - ' +  COALESCE(CAST(ERROR_LINE() AS varchar(200)), ' ')
			+ ' ; ErrorMessage - ' +  COALESCE(CAST(ERROR_MESSAGE() AS varchar(200)), ' ') 
			AS [message]
		, GETDATE() AS [logCreated]; 
END

The code below allows you to use the error log

In [ ]:
-- This belongs at the top of the script
DECLARE @scriptName varchar(200) = (SELECT OBJECT_SCHEMA_NAME(@@PROCID) + '.' + OBJECT_NAME(@@PROCID));

-- Use the following code block to wrap commands you are executing
BEGIN TRY
    -- Replace the following RAISERROR with your code
    RAISERROR ('Error raised in TRY block.', -- Message text.  
        16, -- Severity.  
        1 -- State.  
        );
END TRY  
BEGIN CATCH  
    EXECUTE [dbo].[LogError] @hasBatchId = 1, @customMessage = @scriptName;
END CATCH;

-- Alternatively you can use this to log things that are not necessarily related to a 
-- batch by passing 0 to @hasBatchId
BEGIN TRY
    -- Replace the following RAISERROR with your code
    RAISERROR ('Error raised in TRY block.', -- Message text.  
        16, -- Severity.  
        1 -- State.  
        );
END TRY  
BEGIN CATCH  
    EXECUTE [dbo].[LogError] @hasBatchId = 0, @customMessage = @scriptName;
END CATCH;
